#### Riffle Shuffles:
A riffle shuffle is executed as follows: a deck of cards is split into two equal halves,<br>
with the top half taken in the left hand and the bottom half taken in the right hand.<br>
Next, the cards are interleaved exactly, with the top card in the right half inserted just<br>
after the top card in the left half, the 2nd card in the right half just after the 2nd card<br>
in the left half, etc.<p>
*Note that this process preserves the location of the top and bottom card of the deck<p>

Let s(n) be the minimum number of consecutive riffle shuffles needed to restore a deck<br>
of size n to its original configuration, where n is a positive even number.<p>

Amazingly, a standard deck of 52 cards will first return to its original configuration<br>
after only 8 perfect shuffles, so s(52)=8. It can be verified that a deck of 86 cards will<br>
also return to its original configuration after exactly 8 shuffles, and the sum of all values<br>
of n that satisfy s(n)=8 is 412.<p>

Find the sum of all values of n that satisfy s(n)=60.



#### Solution:

First, I wrote out the rotations for a few of small deck sizes. Each time, I only followed<br>
the second card in the deck. It became clear that after each shuffle the 2 card had displaced<br>
by a power of 2 modulo the deck size. To get back to the original configuration would then<br>
require $2^{deckSize} - 1$ shuffles. Next, I wrote a function to test this for decks which return<br>
in 8 steps and not fewer. This required finding which factors of 8 were troublesome.<p>
Of course, any deck size larger than 256 will not divide 256 and so is a fine stopping point.<p>

In [2]:
simpler = f 2 0
  where
    f 258 acum = acum
    f n acum | dd n && cc n = f (n+2) (n+acum)
             | otherwise = f (n+2) acum

    cc n = mod 16 (n-1) /= 1
    dd n = mod 256 (n-1) == 1

Lo and behold, it landed the correct solution of 412.

Unfortunately, when I extended this to the cycles of length 60 the time complexity blew up linearly and my algorithm would only finish in 3000 years. Over lunch it occurred to me to look at the prime factorization for $2^{60}-1$ and it's troublesome factors. When I did, It became obvious that the total space could be reduced to 13 binary dimensions and that all common factors could be found bitwise.<p>

#### The insight:

Rather than querying every number between `1` and `1152921504606846975`, I can<br>
consider only those numbers which are factors of $2^{60}-1$. These factors can be<br>
generated directly by considering linear combinations of the prime factorization<br>
for $2^{60}-1$. Importing my favorite primes library, I proceed to get the following:<p>
`[(3,2),(5,2),(7,1),(11,1),(13,1),(31,1),(41,1),(61,1),(151,1),(331,1),(1321,1)]`<p>

Since almost every factor is unique, the list can nearly be interpreted as<br>
a basis for a vector space of factors. The repeated 3's and 5's do provide<br>
for a few hoops to jump through, but these problems are easily surmounted.<br>

#Todo: Reason through the bitwise operations#


In [10]:
import Data.Bits

euler622 = sum.remSort $ map ((+ 1).eval) goodBits

mapping = [3,3,5,5,7,11,13,31,41,151,331,1321,61]

goodBits :: [Integer]
goodBits = [n | n<-[1..2^13-1], bitCond n]
  where
    cond b = all (\c -> b .&. c /= b) 
    bitCond b | b <= 85   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89,87,86,85]
              | b <= 86   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89,87,86]
              | b <= 87   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89,87]
              | b <= 89   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89]
              | b <= 90   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90]
              | b <= 91   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91]
              | b <= 421  = cond b [1715,1714,1713,656,430,429,426,425,422,421]
              | b <= 422  = cond b [1715,1714,1713,656,430,429,426,425,422]
              | b <= 425  = cond b [1715,1714,1713,656,430,429,426,425]
              | b <= 426  = cond b [1715,1714,1713,656,430,429,426]
              | b <= 429  = cond b [1715,1714,1713,656,430,429]
              | b <= 430  = cond b [1715,1714,1713,656,430]
              | b <= 656  = cond b [1715,1714,1713,656]
              | b <= 1713 = cond b [1715,1714,1713]
              | b <= 1714 = cond b [1715,1714]
              | b <= 1715 = cond b [1715]
              | otherwise = True

bitify [] = 0
bitify (n:ns) = n + bitify (map (* 2) ns)

listify 0 = [] 
listify n = mod n 2 :listify (div n 2)

eval bit = f (listify bit) mapping
  where
    f [] m  = 1
    f bs [] = 1
    f (b:bs) (m:ms) | b == 1 = m * (f bs ms)
                    | otherwise = f bs ms

-- dot product for lists
instance Num a => Num [a] where
  (*) as [] = []
  (*) [] as = []
  (*) (a:as) (b:bs) = a*b : as * bs

-- removes duplicates while sorting
remSort :: Ord a => [a] -> [a]
remSort [] = []
remSort [a] = [a]
remSort (a:as) = (remSort.smaller) (a:as) ++ [a] ++ (remSort.larger) (a:as)
  where
    smaller (x:xs) = filter (< x) xs
    larger (x:xs) = filter (> x) xs

<interactive>:1:1: error:
    • Variable not in scope: main
    • Perhaps you meant ‘min’ (imported from Prelude)

In [12]:
euler622


3010983666182123972